<a href="https://colab.research.google.com/github/flamemeister/QazQa/blob/main/Test_Kaz_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install sentencepiece
!pip install nltk

In [ ]:
import pandas as pd
import unicodedata

import torch
from torch.utils.data import DataLoader

from transformers import T5Tokenizer, T5TokenizerFast
from transformers import T5ForConditionalGeneration

from tqdm import tqdm
import sentencepiece

from sklearn.model_selection import train_test_split 
from datetime import datetime

from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.meteor_score import meteor_score

max_text_length   = 128

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/Data/kz-qa/kz-t5')
model     = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Data/kz-qa/kz-t5')
model.load_state_dict(torch.load("/content/drive/MyDrive/Data/kz-qa/t5-kaz-small-qa_30.pt"))

gold = pd.read_csv("/content/drive/MyDrive/Data/kz-qa/gold3.csv", sep=';')

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained('google/mt5-base')
# model     = T5ForConditionalGeneration.from_pretrained('google/mt5-base')

In [ ]:
gold

,context,question,answer
0,"Алтай – Ресей, Моңғолия, Қазақстан және Қытай ...",Алтай тау жүйесі Қазақстанның қай жерінде орна...,Шығыс Қазақстан облысының аумағында.
1,Алтай - көптеген тау шаңғысы курорттары бар әй...,Алтай дегеніміз не?,Шаңғы курорттары бар туристік аймақ.
2,"Қазақстанның климаты континенттік, қысы суық, ...",Қазақстанға қандай климат тән?,"Қысы суық, жазы ыстық континенталды климат."
3,Ақбөкен – Орталық Азияда таралған тау ешкісіні...,Ақбөкендер деген не?,Орта Азияда таралған тау ешкілерінің бір түрі.
4,Домбыра – Қазақстанда және Орталық Азияның бас...,Домбыра дегеніміз не?,Қазақстанда және Орталық Азияның басқа елдерін...
5,"Астана – Қазақстанның елордасы, еліміздің солт...",Қазақстанның астанасы қай жерде орналасқан?,Еліміздің солтүстік бөлігінде.
6,Құрғақ климат пен жауын-шашынның аз белсенділі...,Климаттың Қазақстан ауыл шаруашылығына әсері ?,Құрғақ климат және жауын-шашынның аз белсенділ...
7,"Ертіс – Ресей, Қытай және Қазақстан жерін басы...",Ертіс өзені қай елдер арқылы өтеді?,"Ресей, Қытай және Қазақстан."
8,Павлодар — Қазақстандағы Ертіс өзенінің жағасы...,Павлодар қаласы қай жерде орналасқан?,Ертіс өзенінің жағасында.
9,Шымкент — Қазақстанның оңтүстігінде орналасқан...,Қазақстанның оңтүстігіндегі ең үлкен қала?,Шымкент.


In [ ]:
cuda = torch.cuda.is_available()
device = torch.device('cuda') if cuda else torch.device('cpu')
model.to(device)
model.eval()
def test(context, question):
    encoded = tokenizer.encode_plus(context, 
                                    question, 
                                    max_length=max_text_length, 
                                    pad_to_max_length=True, 
                                    truncation=True, 
                                    return_tensors="pt")
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)
    with torch.no_grad():
        output = model.generate(input_ids=input_ids,
                                attention_mask=attention_mask,
                                max_length=max_text_length)
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in output]

In [ ]:
def clean_text(text):
  return(text.replace("<pad>", "").replace("</s>", ""))

In [ ]:
correct = 0
total   = 0

gold['gen_answer'] = ''
gold['score']      = 0.0

for i, row in gold.iterrows():
    context  = row['context']
    question = row['question']
    answer   = row['answer'].lower()[:-1]
    
    # Combine the context and question into a single string
    gen_answer = ''.join(test(context, question)).lower()
    gen_answer = clean_text(gen_answer)
    gold.at[i, 'gen_answer'] = gen_answer
    # Calculate the BLEU score for the generated answer
    score = sentence_bleu([answer.split()], gen_answer.split(), weights=(1, 0, 0))
    gold.at[i, 'score'] = round(score, 4)
    
    # Check if the BLEU score is above a threshold to determine if the generated answer is correct
    threshold = 0.4
    if score >= threshold:
        correct += 1
    total += 1

# Calculate the accuracy and print the result
accuracy = correct / total
print(f"Accuracy: {accuracy}")


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.

Accuracy: 0.4782608695652174


In [ ]:
pd.set_option('max_colwidth', 200)

gold[["context", "question", "gen_answer"]]

,context,question,gen_answer
0,"Алтай – Ресей, Моңғолия, Қазақстан және Қытай шекарасындағы тау сілемі. Қазақстанда Алтай Шығыс Қазақстан облысының аумағында орналасқан.",Алтай тау жүйесі Қазақстанның қай жерінде орналасқан?,шығыс қазақстан облысының аумағында
1,"Алтай - көптеген тау шаңғысы курорттары бар әйгілі туристік аймақ. Алтай тау жүйесі Ресей, Моңғолия, Қазақстан және Қытай шекарасында орналасқан.",Алтай дегеніміз не?,тау шаңғысы курорттары
2,"Қазақстанның климаты континенттік, қысы суық, жазы ыстық. Қазақстанның оңтүстігінде климат біршама қалыпты.",Қазақстанға қандай климат тән?,континенттік
3,Ақбөкен – Орталық Азияда таралған тау ешкісінің бір түрі. Ақбөкендер биік таулы жайылымдарда мекендейді және тау жануарларының ең төзімді түрлерінің бірі болып табылады.,Ақбөкендер деген не?,тау жануарларының ең төзімді түрлерінің бірі болып табылады
4,Домбыра – Қазақстанда және Орталық Азияның басқа елдерінде кеңінен қолданылатын ұлттық музыкалық аспап.,Домбыра дегеніміз не?,ұлттық музыкалық аспап
5,"Астана – Қазақстанның елордасы, еліміздің солтүстік бөлігінде орналасқан. Астана әлемдегі ең жылдам дамып келе жатқан қалалардың бірі және заманауи сәулеттік көрікті жерлерімен танымал.",Қазақстанның астанасы қай жерде орналасқан?,солтүстік
6,Құрғақ климат пен жауын-шашынның аз белсенділігі еліміздің кейбір аймақтарында ауыл шаруашылығының дамуына кедергі келтіреді. Қазақстанның көптеген аймақтары егіншіліктің дамуына кедергі келтіреті...,Климаттың Қазақстан ауыл шаруашылығына әсері ?,құрғақ климат және жауын-шашынның аз белсенділігі
7,"Ертіс – Ресей, Қытай және Қазақстан жерін басып өтетін өзен. Қазақстанда Ертіс – Обь өзенінің ең үлкен саласы.",Ертіс өзені қай елдер арқылы өтеді?,"ресей, қытай және қазақстан"
8,Павлодар — Қазақстандағы Ертіс өзенінің жағасында орналасқан қала. Қалада көптеген өнеркәсіп кәсіпорындары бар.,Павлодар қаласы қай жерде орналасқан?,ертіс өзені
9,Шымкент — Қазақстанның оңтүстігінде орналасқан қала. Шымкент – Қазақстанның оңтүстігіндегі ең үлкен қала.,Қазақстанның оңтүстігіндегі ең үлкен қала?,шымкент


In [ ]:
context = """Қар деп атмосфералық жауын-шашынды айтады. Қар ұсақ мұз кристалдарынан тұрады. 
Қазақстан аймағында қар қазан-қараша айларынан бастап түседі. 
Қазақстандағы қар жамылғысының қалыңдығы 50-60 см-ге жетеді. Қардың түсі ақ."""

question_1 = "Қар дегеніміз не?"
question_2 = "Қар неден тұрады?"
question_3 = "Қардың түсі қандай?"
question_4 = "Қазақстанда қар жамылғысының қалыңдығы қанша?"

answer_1 = ''.join(test(context, question_1))
answer_2 = ''.join(test(context, question_2))
answer_3 = ''.join(test(context, question_3))
answer_4 = ''.join(test(context, question_4))


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(context, "\n")

print(question_1, "---> ", answer_1)
print(question_2, "---> ", answer_2)
print(question_3, "---> ", answer_3)
print(question_4, "---> ", answer_4)

Қар деп атмосфералық жауын-шашынды айтады. Қар ұсақ мұз кристалдарынан тұрады. 
Қазақстан аймағында қар қазан-қараша айларынан бастап түседі. 
Қазақстандағы қар жамылғысының қалыңдығы 50-60 см-ге жетеді. Қардың түсі ақ. 

Қар дегеніміз не? --->  атмосфералық жауын-шашынды
Қар неден тұрады? --->  ұсақ мұз кристалдары
Қардың түсі қандай? --->  ақ
Қазақстанда қар жамылғысының қалыңдығы қанша? --->  5060


In [ ]:
x = ['Шығыс', 'Қазақстан', 'облысының', 'аумағында'] 

y = ['Шығыс', 'Қазақстан', 'облысының', 'аумағында']

print(x[1]==y[1])
print(sentence_bleu([x], y, weights=(0.6, 0.4)))


True
1.0
